In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer() #Initialize lemmatizer
from nltk.corpus import wordnet
from collections import Counter
import itertools

>* We are going to use email data from program-l mailing list to build edges and nodes for the graph.
>* https://www.freelists.org/archive/program-l
> * Let's read file using `pd.read_csv` 

In [2]:
data=pd.read_csv('thread_sample.csv')

>* In `data` dataframe, we have six columns, each representing as follows:
    
    * `thread_id` : unique id for each thread
    * `thread_name` : the first subject of the email
    * `body` : the content of the email 
    * `account` : the email account of the sender 
    * `url` : the url of the email
    * `date` : the date of the email 

### 1. Regular Expression (RegEx) (2pt)

#### 1-1. RegEx to extract usernames (1pt)

In [4]:
data['account'].iloc[0]

"['chojiro1990' 'soronel.haetir' 'chojiro1990' 'soronel.haetir'\n 'chojiro1990' 'soronel.haetir' 'chojiro1990' 'soronel.haetir'\n 'chojiro1990']"

In [3]:
#YOUR CODE HERE

#### 1-2. RegEx to extract date (1pt)

In [8]:
data['date'].iloc[0]

"['2013-06-05T09:50:54.000000000' '2013-06-05T15:42:35.000000000'\n '2013-06-05T16:28:36.000000000' '2013-06-05T19:52:01.000000000'\n '2013-06-06T09:33:53.000000000' '2013-06-06T15:31:31.000000000'\n '2013-06-06T19:15:11.000000000' '2013-06-06T20:04:44.000000000'\n '2013-06-07T05:05:21.000000000']"

In [7]:
#YOUR CODE HERE

### 2. Pandas and Text mining (6pt)

#### 2-1. Count the number of users involved in the email conversation (1pt)

In [11]:
#YOUR CODE HERE

#### 2-2. Lowercase the body column (1pt)

In [14]:
#YOUR CODE HERE

#### 2-3. Removing stopwords (1pt)

In [17]:
#YOUR CODE HERE

#### 2-4. Tokenization and removing punctuations (1pt)

In [21]:
#YOUR CODE HERE

#### 2-5. Lemmatization (1pt)

In [26]:
#YOUR CODE HERE

In [27]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'): #ADJECTIVE
        return wordnet.ADJ
    elif nltk_tag.startswith('V'): #VERN
        return wordnet.VERB
    elif nltk_tag.startswith('N'): #NOUN        
        return wordnet.NOUN
    elif nltk_tag.startswith('R'): #ADVERB
        return wordnet.ADV
    else:          
        return None

In [28]:
def lemmatize_sentence(sentence):
    # Tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    # Tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged) 
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # If no tag was found, then use the word as is
            lemmatized_sentence.append(word)
        else:        
            # Else use the tag to lemmatize the word
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

#### 2-6. Count the frequency of unigrams (1pt)

In [31]:
#YOUR CODE HERE

### 3. Building edges (2pt)

#### 3-1. Combination

In [33]:
#YOUR CODE HERE

#### 3-2. Removing self-loop (1pt)

In [36]:
#### YOUR CODE HERE